In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Define banking book
bank_data = {
    'ProductID': ['L1', 'L2', 'D1', 'D2'],
    'ProductType': ['Fixed-Rate Loan', 'Floating-Rate Loan', 'Floating-Rate Deposit', 'Demand Deposit'],
    'Currency': ['CAD', 'CAD', 'CAD', 'CAD'],
    'Notional': [100_000_000, 50_000_000, 120_000_000, 30_000_000],
    'RateType': ['Fixed', 'Variable', 'Variable', 'Fixed'],
    'Rate': [0.04, 0.03, 0.025, 0.0],
    'RepricingDate': ['2030-04-21', '2025-07-21', '2025-07-21', None],
    'MaturityDate': ['2030-04-21', '2026-04-21', '2026-04-21', None],
    'TimeBucket': ['5Y', '3M', '3M', 'Non-Repricing']
}


In [3]:
# Create DataFrame
df = pd.DataFrame(bank_data)

In [4]:
df

,ProductID,ProductType,Currency,Notional,RateType,Rate,RepricingDate,MaturityDate,TimeBucket
0,L1,Fixed-Rate Loan,CAD,100000000,Fixed,0.040,2030-04-21,2030-04-21,5Y
1,L2,Floating-Rate Loan,CAD,50000000,Variable,0.030,2025-07-21,2026-04-21,3M
2,D1,Floating-Rate Deposit,CAD,120000000,Variable,0.025,2025-07-21,2026-04-21,3M
3,D2,Demand Deposit,CAD,30000000,Fixed,0.000,None,None,Non-Repricing


In [5]:
# Define EBA shock scenarios
scenarios = {
    'Parallel Up': {'short': 0.02, 'long': 0.02},
    'Parallel Down': {'short': -0.02, 'long': -0.02},  # Floored at 0% for short rates
    'Steepener': {'short': -0.01, 'long': 0.01},
    'Flattener': {'short': 0.01, 'long': -0.01},
    'Short Rates Up': {'short': 0.025, 'long': 0.0},
    'Short Rates Down': {'short': -0.025, 'long': 0.0}  # Floored at 0%
}


In [6]:
# Base yield curve (flat at 2%)
base_rate = 0.02

In [7]:
# Helper functions
def pvifa(r, n):
    """Calculate Present Value Interest Factor for an Annuity."""
    #r: The interest rate per period (as a decimal, e.g., 5% = 0.05).
    #n: The number of periods (e.g., years or payment periods).
    #(1 + r): Represents the growth factor for one period.
    #(1 + r) ** -n: Calculates the inverse of the compound interest factor, i.e., 1(1+r)n\frac{1}{(1 + r)^n}\frac{1}{(1 + r)^n}
    #, which discounts the final period back to the present.
    #1 - (1 + r) ** -n: Computes the sum of the geometric series of discounted values for the annuity.
    #/ r: Divides by the interest rate to get the PVIFA factor.
    
    if r == 0:
        return n
    return (1 - (1 + r) ** -n) / r

In [8]:
def pvif(r, n):
    """Calculate Present Value Interest Factor for a single sum."""
    return (1 + r) ** -n

In [9]:
def calculate_eve(df, discount_rate):
    """Calculate EVE for assets and liabilities."""
    pv_assets = 0
    pv_liabilities = 0

    for idx, row in df.iterrows():
        notional = row['Notional']
        rate = row['Rate']
        maturity = 5 if row['TimeBucket'] == '5Y' else 0.25 if row['TimeBucket'] == '3M' else 0

        if row['ProductType'] in ['Fixed-Rate Loan', 'Floating-Rate Loan']:
            if row['TimeBucket'] == '5Y':
                # Fixed-rate loan: coupon + principal
                coupon = notional * rate
                pv_coupon = coupon * pvifa(discount_rate, 5)
                pv_principal = notional * pvif(discount_rate, 5)
                pv_assets += pv_coupon + pv_principal
            else:
                # Floating-rate loan: PV ~ notional (short-term)
                pv_assets += notional
        elif row['ProductType'] == 'Floating-Rate Deposit':
            # Floating-rate deposit: PV ~ notional
            pv_liabilities += notional
        elif row['ProductType'] == 'Demand Deposit':
            # Non-interest-bearing: PV = notional
            pv_liabilities += notional

    eve = pv_assets - pv_liabilities
    return eve

In [10]:
def calculate_nii(df, short_rate):
    """Calculate NII over 1-year horizon."""
    income = 0
    expense = 0

    for idx, row in df.iterrows():
        notional = row['Notional']
        rate = row['Rate']

        if row['ProductType'] == 'Fixed-Rate Loan':
            income += notional * rate
        elif row['ProductType'] == 'Floating-Rate Loan':
            income += notional * (short_rate + 0.01)  # Spread +1%
        elif row['ProductType'] == 'Floating-Rate Deposit':
            expense += notional * (short_rate + 0.005)  # Spread +0.5%
        elif row['ProductType'] == 'Demand Deposit':
            expense += 0

            
#A spread is an additional percentage added to account for factors such as:
#Credit Risk, Profit Margin, Operational Costs, Market Conventions , Liquidity and Funding Costs            
            
    nii = income - expense
    return nii

In [11]:
# Base case calculations
base_eve = calculate_eve(df, base_rate)
base_nii = calculate_nii(df, base_rate)
print(f"Base EVE: ${base_eve:,.2f}")
print(f"Base NII: ${base_nii:,.2f}")

Base EVE: $9,426,919.02
Base NII: $2,500,000.00


In [12]:
# Simulate scenarios
results = []
for scenario, shocks in scenarios.items():
    # Adjust rates
    short_rate = max(base_rate + shocks['short'], 0)  # Floor at 0%
    long_rate = base_rate + shocks['long']

    # Calculate shocked EVE and NII
    shocked_eve = calculate_eve(df, long_rate)
    shocked_nii = calculate_nii(df, short_rate)

    # Calculate deltas
    delta_eve = shocked_eve - base_eve
    delta_nii = shocked_nii - base_nii

    results.append({
        'Scenario': scenario,
        'Base Rate':base_rate,
        'Short Rate':round(short_rate, 2),
        'Long Rate':long_rate,
        'ΔEVE': delta_eve,
        'ΔEVE (%)': delta_eve / 50_000_000 * 100,  # % of Tier 1 Capital ($50M)
        'ΔNII': delta_nii,
        'ΔNII (%)': delta_nii / 50_000_000 * 100
    })


In [13]:
# Create results DataFrame
results_df = pd.DataFrame(results)
print("\nIRRBB Stress Test Results:")
print(results_df.to_string(index=False))
results_df.to_csv('IRRBB_Result.csv', index=False)


IRRBB Stress Test Results:
        Scenario  Base Rate  Short Rate  Long Rate          ΔEVE   ΔEVE (%)       ΔNII  ΔNII (%)
     Parallel Up       0.02        0.04       0.04 -9.426919e+06 -18.853838 -1400000.0      -2.8
   Parallel Down       0.02        0.00       0.00  1.057308e+07  21.146162  1400000.0       2.8
       Steepener       0.02        0.01       0.03 -4.847212e+06  -9.694424   700000.0       1.4
       Flattener       0.02        0.03       0.01  5.133375e+06  10.266749  -700000.0      -1.4
  Short Rates Up       0.02        0.04       0.02  0.000000e+00   0.000000 -1750000.0      -3.5
Short Rates Down       0.02        0.00       0.02  0.000000e+00   0.000000  1400000.0       2.8


Steps to Comply with IRRBB
To bring ΔEVE and ΔNII within acceptable regulatory limits, the bank needs to reduce its interest rate risk exposure. 
1. Reduce EVE Sensitivity
    EVE measures the long-term impact of rate changes on the present value of all cash flows. 
    Large ΔEVE indicates a duration mismatch between assets and liabilities.

    Adjust Asset-Liability Duration:
        Shorten Asset Duration: If assets have a longer duration than liabilities  the bank loses 
        Lengthen Liability Duration: Issue longer-term fixed-rate deposits or debt to better match asset durations.
    Use Interest Rate Swaps: Enter into pay-fixed, receive-floating swaps to hedge against rising rates. 
    Increase Capital Buffer:increasing Tier 1 capital can bring the ratio below the threshold

2. Reduce NII Sensitivity
    NII measures the short-term impact of rate changes on earnings. 
    Large ΔNII indicates over-reliance on floating-rate instruments with narrow spreads.

    Widen Spreads:
        The current spreads result in a narrow net interest margin (0.5%). Increasing the loan spread can stabilize NII by  providing a larger buffer against rate changes.

    Shift to Fixed-Rate Products:Increase the proportion of fixed-rate loans or fixed-rate deposits.

